In [1]:
import pandas as pd
import numpy as np

import time
import datetime

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [2]:
features = pd.read_csv('features/features.csv', index_col='match_id')
features.drop('duration radiant_win tower_status_radiant tower_status_dire barracks_status_radiant barracks_status_dire'.split(' '), inplace=True, axis=1)

In [3]:
features_test = pd.read_csv('features_test/features_test.csv', index_col='match_id')

In [4]:
features = features.fillna(0)
features_test = features_test.fillna(0)

In [5]:
X_train , Y_train = features.as_matrix(), pd.read_csv('features/features.csv', index_col='match_id')['radiant_win'].as_matrix()
X_test = features_test.as_matrix()

1.Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [23]:
def testing(X_train, Y_train, c=[10.0 ** i for i in range(-9, 8)]):
    grid = {'C': c}
    cv = KFold(n_splits=5, shuffle=True, random_state=241)
    clf = LogisticRegression(penalty='l2',random_state=241)
    
    gs = GridSearchCV(clf, scoring='roc_auc', param_grid=grid, cv=cv, return_train_score=True)
    gs.fit(X_train,Y_train)
    
    means = gs.cv_results_['mean_test_score']
    stds = gs.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    
def testing_cros(X_train, Y_train, c=[10.0 ** i for i in range(-9, 8)]):     
    start_time = datetime.datetime.now()
    scores=[]
    
    for x in c:
        clf = LogisticRegression(penalty='l2', C=x, random_state=241)
        scores.append(np.array(cross_val_score(clf, X_train, Y_train, scoring='roc_auc', cv=cv)))
    scores=np.array(scores)
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    print (c[np.argmax([np.mean(sc) for sc in scores])])